In [9]:
import pandas as pd
import numpy as np
from copy import deepcopy
from sklearn import metrics as mt 
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation, Input
from tensorflow.keras.models import Model

df = pd.read_csv('Video_games_esrb_rating.csv')

# map the esrb string values to integers for our target
rating = []

for i in df['esrb_rating']:
    if(i == 'E'):
        rating.append(0)
    elif(i == 'ET'):
        rating.append(1)
    elif(i == 'T'):
        rating.append(2)
    else:
        rating.append(3)
        
del df['title'] #get rid of unnecessary data
del df['console']
del df['esrb_rating'] #get rid of original column since its been mapped now

from sklearn.model_selection import ShuffleSplit

y = np.array(rating).T

#remaining columns in dataset are used for X
X = df.to_numpy()

#set up shuffle split using example
num_cv_iterations = 10
num_instances = len(y)
cv_object = ShuffleSplit(n_splits = num_cv_iterations, test_size = 0.2)
for train_index, test_index in cv_object.split(X,y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

num_features = X_train.shape[1]
input_tensor = Input(shape=(num_features,))
x = Dense(units=10, activation='relu')(input_tensor)
x = Dense(units=5, activation='tanh')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=input_tensor, outputs=predictions)
model.compile(optimizer='sgd',
              loss='mean_squared_error',
              metrics=['accuracy'])

model.summary()


Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 31)]              0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                320       
_________________________________________________________________
dense_10 (Dense)             (None, 5)                 55        
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 6         
Total params: 381
Trainable params: 381
Non-trainable params: 0
_________________________________________________________________


In [10]:
%%time

model.fit(X_train, y_train, epochs=10, batch_size=50, verbose=1)
from sklearn import metrics as mt
yhat_proba = model.predict(X_test)
yhat = np.round(yhat_proba)
print(mt.confusion_matrix(y_test,yhat))
print(mt.classification_report(y_test,yhat))

Epoch 1/10
31/31 [==============================] - 0s 812us/step - loss: 2.0148 - accuracy: 0.2071
Epoch 2/10
31/31 [==============================] - 0s 837us/step - loss: 1.8417 - accuracy: 0.2111
Epoch 3/10
31/31 [==============================] - 0s 670us/step - loss: 1.7218 - accuracy: 0.2111
Epoch 4/10
31/31 [==============================] - 0s 645us/step - loss: 1.6427 - accuracy: 0.2111
Epoch 5/10
31/31 [==============================] - 0s 582us/step - loss: 1.5893 - accuracy: 0.2111
Epoch 6/10
31/31 [==============================] - 0s 548us/step - loss: 1.5525 - accuracy: 0.2111
Epoch 7/10
31/31 [==============================] - 0s 549us/step - loss: 1.5255 - accuracy: 0.2111
Epoch 8/10
31/31 [==============================] - 0s 645us/step - loss: 1.5059 - accuracy: 0.2111
Epoch 9/10
31/31 [==============================] - 0s 581us/step - loss: 1.4912 - accuracy: 0.2111
Epoch 10/10
31/31 [==============================] - 0s 586us/step - loss: 1.4794 - accuracy: 0.2111

C:\Users\infin\.conda\envs\mlenv2020\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
